# 0. Imports and data loading

In [3]:
'''
All packages and modules are imported in the "functions.py" file located next to this notebook

Additionally a number of code chunks employed in previous versions of this notebook have been
defined into functions and placed in that same file in order to simplify the exposition of the 
main points and results of the assignment. For further insight on the code behind the actions
here executed this "functions.py" file can be visited, there all the details can be found.

IMPORTANT: for the appropriate functioning of this notebook it must be placed in the same 
directory as "functions.py"
'''

from functions import *

In [4]:
features = np.load('data/data.npy')
labels = np.load('data/labels.npy')

# 1. Data preprocessing



### 1.1 Splitting Data

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

### Extra: Easy going here! - Sample restriction 😎🤙🌴  \#prayforkernel 🙏

In [6]:
#############################################
#             CONTROL PANEL

# Sample restriction
restriction = True
address_vp_imbalance = True
proportion = 0.15
#                                           
#############################################

In [7]:
if restriction == True:
    
    if address_vp_imbalance == True:
        
        restricted_sample_train, restricted_sample_labels_train = restrict_sample_vp_imb(proportion , X_train, y_train)
        restricted_sample_val, restricted_sample_labels_val = restrict_sample_vp_imb(proportion , X_val, y_val)
        restricted_sample_test, restricted_sample_labels_test = restrict_sample_vp_imb(proportion , X_test, y_test)
        
    else:

        restricted_sample_train, restricted_sample_labels_train = restrict_sample(proportion , X_train, y_train)
        restricted_sample_val, restricted_sample_labels_val = restrict_sample(proportion , X_val, y_val)
        restricted_sample_test, restricted_sample_labels_test = restrict_sample(proportion , X_test, y_test)

In [8]:
if restriction == True:
    
    ############################
    # > RESTRICTION  SUMMARY < #
    ############################
    print("SAMPLE RESTRICTION SUMMARY")
    print("")
    print("")
    print("ORIGINAL TRAINING SAMPLE")
    print(F"Original number of instances: {len(X_train)}")
    print(f"Original instance distribution by class: \n {pd.Series(y_train).value_counts()}")
    print("")
    print("RESTRICTED TRAINING SAMPLE")
    print(f"Number of instances in restricted sample: {len(restricted_sample_train)}")
    print(f"Instance distribution by class in restricted sample: \n {pd.Series(restricted_sample_labels_train).value_counts()}")
    
    X_train = np.array(restricted_sample_train)
    y_train = np.array(restricted_sample_labels_train)
    X_val = np.array(restricted_sample_val)
    y_val = np.array(restricted_sample_labels_val)
    X_test = np.array(restricted_sample_test)
    y_test = np.array(restricted_sample_labels_test)
    
else:
    print("SAMPLE RESTRICTION WAS NOT CONDUCTED")

SAMPLE RESTRICTION SUMMARY


ORIGINAL TRAINING SAMPLE
Original number of instances: 4147
Original instance distribution by class: 
 Bacterial Pneumonia       1689
Viral Pneumonia           1182
No Pneumonia (healthy)     963
COVID-19                   313
dtype: int64

RESTRICTED TRAINING SAMPLE
Number of instances in restricted sample: 1047
Instance distribution by class in restricted sample: 
 Viral Pneumonia           337
COVID-19                  313
Bacterial Pneumonia       253
No Pneumonia (healthy)    144
dtype: int64


### 1.2 Normalize Data

In [9]:
# Float conversion to allow normalization
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_val=X_val.astype('float32')

# Normalization 
X_train=X_train/255.0
X_test=X_test/255.0
X_val=X_val/255.0

# Compute the mean and standard deviation of the training set for standardization (NOT USED)
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

# Standardization (NOT USED)
X_train_norm = (X_train - train_mean) / train_std
X_val_norm = (X_val - train_mean) / train_std
X_test_norm = (X_test - train_mean) / train_std


### 1.3 Categorical encoding

In [10]:
# Define a dictionary that maps each category to a numerical value
label_map = {"Bacterial Pneumonia": 0, "Viral Pneumonia": 1, "No Pneumonia (healthy)": 2, "COVID-19": 3}

# Encode the categorical labels as numerical values using the label map
y_train_encoded = np.vectorize(label_map.get)(y_train)
y_val_encoded = np.vectorize(label_map.get)(y_val)
y_test_encoded = np.vectorize(label_map.get)(y_test)

# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical(y_val_encoded, num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)



# 2. Baseline Model

### 2.1 Create baseline model

In [14]:

from tensorflow import keras

def build_baseline_model():
    model = keras.Sequential([
        # Convolutional layers
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(156, 156, 3)),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Dense layers
        keras.layers.Flatten(),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(4, activation='softmax')
    ])

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


baseline_model = build_baseline_model()

# Train the model for 10 epochs with a batch size of 32
history = baseline_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_norm, y_val_onehot)
)


2023-03-04 09:02:23.053470: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-04 09:02:23.055078: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-04 09:02:23.056046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (be39edd05863): /proc/driver/nvidia/version does not exist
2023-03-04 09:02:23.064725: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 09:02:23.682938: W tensorflow/tsl/framework/cpu_allocator_im

Epoch 1/10


2023-03-04 09:02:27.128376: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 194281472 exceeds 10% of free system memory.
2023-03-04 09:02:29.578815: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 194281472 exceeds 10% of free system memory.



 1/40 [..............................] - ETA: 4:38 - loss: 1.3875 - accuracy: 0.2500

2023-03-04 09:02:31.498282: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 194281472 exceeds 10% of free system memory.
2023-03-04 09:02:33.262060: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 194281472 exceeds 10% of free system memory.


40/40 [==============================] - 156s 4s/step - loss: 1.1333 - accuracy: 0.5354 - val_loss: 0.9355 - val_accuracy: 0.6280
Epoch 2/10
40/40 [==============================] - 147s 4s/step - loss: 0.8364 - accuracy: 0.6630 - val_loss: 0.9171 - val_accuracy: 0.6469
Epoch 3/10
40/40 [==============================] - 145s 4s/step - loss: 0.7316 - accuracy: 0.7126 - val_loss: 0.8077 - val_accuracy: 0.6801
Epoch 4/10
40/40 [==============================] - 142s 4s/step - loss: 0.6202 - accuracy: 0.7567 - val_loss: 0.8000 - val_accuracy: 0.6943
Epoch 5/10
40/40 [==============================] - 133s 3s/step - loss: 0.5285 - accuracy: 0.7882 - val_loss: 0.9739 - val_accuracy: 0.6706
Epoch 6/10
40/40 [==============================] - 132s 3s/step - loss: 0.5007 - accuracy: 0.7882 - val_loss: 0.7928 - val_accuracy: 0.6777
Epoch 7/10
40/40 [==============================] - 131s 3s/step - loss: 0.3847 - accuracy: 0.8409 - val_loss: 0.9787 - val_accuracy: 0.6872
Epoch 8/10
40/40 [======

In [ ]:
baseline_all_val_acc = np.mean(history.history["val_accuracy"])
baseline_all_val_loss = np.mean(history.history["val_loss"])
baseline_all_train_acc = np.mean(history.history["accuracy"])
baseline_all_train_loss = np.mean(history.history["loss"])
print("BASELINE RESULTS:")
print("-"*len("BASELINE RESULTS:"))
print()
print("**Training**")
print("The average training accuracy among all epochs is: {:.4}".format(baseline_all_train_acc))
print("The average training loss among all epochs is: {:.4}".format(baseline_all_train_loss))
print()
print("**Validation**")
print("The average validation accuracy among all epochs is: {:.4}".format(baseline_all_val_acc))
print("The average validation loss among all epochs is: {:.4}".format(baseline_all_val_loss))

### 2.2 Analyze the performance of the baseline model

In [1]:
import matplotlib.pyplot as plt

def plot_acc_loss(histo):
  ##Plot for the accuracy of the baseline model 
  accuracy_train = histo.history['accuracy']
  accuracy_val = histo.history['val_accuracy']
  plt.plot(accuracy_train, label='training_accuracy')
  plt.plot(accuracy_val, label='validation_accuracy')
  plt.title('ACCURACY OF THE MODEL')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  plt.show()

  ##Plot for the loss of the baseline model 
  loss_train = histo.history['loss']
  loss_val = histo.history['val_loss']
  plt.plot(loss_train, label='training_loss')
  plt.plot(loss_val, label='validation_loss')
  plt.title('LOSS OF MODEL')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()
  return 


##------ploting from BASELINE only 
plot_acc_loss(history)


##----Getting prediction based in the baseline model 
y_pred = baseline_model.predict(X_test_norm) 

def plot_ROC_curve(y_predict,y_test,num_clas): 
  from sklearn.metrics import roc_curve, auc
  import matplotlib.pyplot as plt
  fpr = {}
  tpr = {}
  roc_auc = {}
  #calculating roc for each class
  for i in range(num_clas):
      fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_predict[:,i])
      roc_auc[i] = auc(fpr[i], tpr[i])
  from sklearn.metrics import roc_auc_score
  # calculating micro-average ROC curve and  area
  fpr_micro, tpr_micro, _ = roc_curve(y_test.ravel(), y_predict.ravel())
  roc_auc_micro = roc_auc_score(y_test.ravel(), y_predict.ravel())

  # Compute macro-average ROC curve and  area
  fpr_macro = np.unique(np.concatenate([fpr[i] for i in range(num_clas)]))
  tpr_macro = np.zeros_like(fpr_macro)
  for i in range(num_clas):
      tpr_macro += np.interp(fpr_macro, fpr[i], tpr[i])
  tpr_macro /= num_clas
  roc_auc_macro = auc(fpr_macro, tpr_macro)

  #Plot the ROC curve for each class using matplotlib.pyplot.plot()
  plt.figure(figsize=(10, 5))
  lw = 2
  for i in range(num_clas):
      plt.plot(fpr[i], tpr[i], lw=lw, label='ROC curve of class %d (area = %0.2f)' % (i, roc_auc[i]))
  plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
  plt.plot(fpr_micro, tpr_micro,lw=lw, linestyle='--', label='micro-average ROC curve (area = %0.2f)' % (roc_auc_micro))
  plt.plot(fpr_macro, tpr_macro,lw=lw, linestyle='--', label='macro-average ROC curve (area = %0.2f)' % (roc_auc_macro))
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic of Multiclass')
  plt.legend(loc="lower right")
  plt.show()
  return 


#-------Plot ROC of the baselinemodel 
plot_ROC_curve(y_pred,y_test_onehot, num_classes)


def plot_cm(label_ma,y_predict,y_tes):
  #reversing pred to categorical so to get the labels 
  inverse_label_map = {v: k for k, v in label_ma.items()}  # invert the label_map
  y_pred_decoded_numerical = np.argmax(y_predict, axis=1)
  y_pred_decoded_categorical = np.vectorize(inverse_label_map.get)(y_pred_decoded_numerical)

  #confusion matrix 
  from sklearn.metrics import confusion_matrix
  cm = confusion_matrix(y_tes, y_pred_decoded_categorical)
  classes = np.unique(y_tes)
  # plot the confusion matrix
  fig, ax = plt.subplots()
  im = ax.imshow(cm, interpolation='nearest', cmap='Reds')
  ax.figure.colorbar(im, ax=ax)
  ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), xticklabels=classes, yticklabels=classes, ylabel='True label', xlabel='Predicted label')

  # rotate the labels
  plt.setp(ax.get_xticklabels(), rotation=20, ha="right", rotation_mode="anchor")
  # text annotations like the numbers inside 
  thresh = cm.max() / 2.
  for i in range(cm.shape[0]):
      for j in range(cm.shape[1]):
          ax.text(j, i, format(cm[i, j], 'd'), ha="center", va="center", color="white" if cm[i, j] > thresh else "black")
  plt.show()
  return

##----CM Matrix plot
plot_cm(label_map,y_pred,y_test)


def table_p_r_f1(y_tes,y_predict, label_ma):
  from sklearn.metrics import classification_report
  inverse_label_map = {v: k for k, v in label_ma.items()}  # invert the label_map
  y_pred_decoded_numerical = np.argmax(y_predict, axis=1)
  y_pred_decoded_categorical = np.vectorize(inverse_label_map.get)(y_pred_decoded_numerical)

  print(classification_report(y_test, y_pred_decoded_categorical))

#precision and recall and f1-score, accuracy 
table_p_r_f1(y_test,y_pred,label_map)

NameError: name 'history2' is not defined

# 3. Adapting/fine-tuning the network

In [5]:
#############################################
#             CONTROL PANEL (Fine tuning)

# Sample restriction
restriction = True
address_vp_imbalance = True
proportion = 0.25

# Sample augmentation
augmentation_flip = True                       
augmentation_rotate = True

#                                           
#############################################


# safety mechanism: augmentation and restriction are not meant to be combined
if restriction == True:
    agumentation_flip = False
    augmentation_rotate = False

### [DISABLED] 3.1 Data restriction 

In [10]:
#if restriction == True:
#
#    restricted_sample, restricted_sample_labels = restrict_sample(proportion , X_train, y_train)

In [11]:
#if restriction == True:
#    
#    ############################
#    # > RESTRICTION  SUMMARY < #
#    ############################
#    print("SAMPLE RESTRICTION SUMMARY")
#    print("")
#    print("")
#    print("ORIGINAL SAMPLE")
#    print(F"Original number of instances: {len(X_train)}")
#    print(f"Original instance distribution by class: \n {pd.Series(y_train).value_counts()}")
#    print("")
#    print("RESTRICTED SAMPLE")
#    print(f"Number of instances in restricted sample: {len(restricted_sample)}")
#    print(f"Instance distribution by class in restricted sample: \n {pd.Series(restricted_sample_labels).value_counts()}")
#    
#    X_train = np.array(restricted_sample)
#    y_train = np.array(restricted_sample_labels)
#    
#else:
#    print("SAMPLE RESTRICTION WAS NOT CONDUCTED")

SAMPLE RESTRICTION SUMMARY


ORIGINAL SAMPLE
Original number of instances: 4147
Original instance distribution by class: 
 Bacterial Pneumonia       1689
Viral Pneumonia           1182
No Pneumonia (healthy)     963
COVID-19                   313
dtype: int64

RESTRICTED SAMPLE
Number of instances in restricted sample: 957
Instance distribution by class in restricted sample: 
 Bacterial Pneumonia       422
COVID-19                  313
Viral Pneumonia           295
No Pneumonia (healthy)    240
dtype: int64


### 3.2 [DISABLED] Data Augmentation

In [6]:
#if augmentation_flip == True:
#
#    augmentedfeatures, augmentedlabels = augment_sample(X_train, y_train, a_rotate=augmentation_rotate)

2023-03-06 06:39:41.150174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-06 06:39:41.150258: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-06 06:39:41.150286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (be39edd05863): /proc/driver/nvidia/version does not exist
2023-03-06 06:39:41.150811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#if augmentation_flip == True:
#    
#    ############################
#    # > AUGMENTATION SUMMARY < #
#    ############################
#    
#    print("SAMPLE AUGMENTATION SUMMARY")
#    print("")
#    print("")
#    print("ORIGINAL SAMPLE")
#    print(F"Original number of instances: {len(y_train)}")
#    print(f"Original instance distribution by class: \n {pd.Series(y_train).value_counts()}")
#    print("")
#    print("AUGMENTED SAMPLE")
#    print(f"Number of instances in augmented sample: {len(augmentedlabels)}")
#    print(f"Instance distribution by class in augmented sample: \n {pd.Series(augmentedlabels).value_counts()}")
#    
#    X_train = np.array(augmentedfeatures)
#    y_train = np.array(augmentedlabels)
#
#else:
#    print("SAMPLE AUGMENTATION WAS NOT CONDUCTED")

SAMPLE AUGMENTATION SUMMARY


ORIGINAL SAMPLE
Original number of instances: 4147
Original instance distribution by class: 
 Bacterial Pneumonia       1689
Viral Pneumonia           1182
No Pneumonia (healthy)     963
COVID-19                   313
dtype: int64

AUGMENTED SAMPLE
Number of instances in augmented sample: 11320
Instance distribution by class in augmented sample: 
 Bacterial Pneumonia       3884
Viral Pneumonia           2718
COVID-19                  2504
No Pneumonia (healthy)    2214
dtype: int64


### 3.3 [DISABLED] Categorical encoding of new labels

In [14]:
## Define a dictionary that maps each category to a numerical value
#label_map = {"Bacterial Pneumonia": 0, "Viral Pneumonia": 1, "No Pneumonia (healthy)": 2, "COVID-19": 3}
#
## Encode the categorical labels as numerical values using the label map
#y_train_encoded = np.vectorize(label_map.get)(y_train)
#y_val_encoded = np.vectorize(label_map.get)(y_val)
#y_test_encoded = np.vectorize(label_map.get)(y_test)
#
## Convert the numerical labels to one-hot encoded format
#num_classes = 4
#y_train_onehot = keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
#y_val_onehot = keras.utils.to_categorical(y_val_encoded, num_classes=num_classes)
#y_test_onehot = keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)

### 3.4 Network fine tuning

In [ ]:
#After some tryouts, Nadam reported an extra 0-2% val_accuracy in our baseline model vs Adam
#So we will find the best learning rate for this optimizer:

from tensorflow import keras
import keras.backend as K
import matplotlib.pyplot as plt

def lr_schedule(epoch, initial_lr, final_lr, total_epochs):
    """
    calculates the learning rate for each epoch based on the initial learning rate, final learning rate, and total number of epochs
    """
    lr = initial_lr + (final_lr - initial_lr) * (epoch / float(total_epochs))
    return lr

def plot_lr_schedule(initial_lr, final_lr, total_epochs):
    lr = [lr_schedule(epoch, initial_lr, final_lr, total_epochs) for epoch in range(total_epochs)]
    plt.plot(lr, history.history['val_loss'])
    plt.xlabel('Learning Rate')
    plt.ylabel('Validation Loss')
    plt.title('Learning Rate Schedule')
    plt.show()

# Adding a Lr Scheduler to check the learning rate evolution during training and to avoid overfitting
initial_lr = 0.001
final_lr = 0.01
baseline_epochs = 10

lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch, initial_lr, final_lr, baseline_epochs))

baseline_model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

# Train the model for 4 epochs with a batch size of 32
history = baseline_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=baseline_epochs,
    validation_data=(X_val_norm, y_val_onehot),
    callbacks=[lr_scheduler]
)


In [ ]:
# Rule of thumb: optimal will be a bit lower than when lr starts climbing, usually 10 times lower the climb up point (around 0.005)
plot_lr_schedule(initial_lr, final_lr, baseline_epochs)

In [ ]:
"""from keras import backend as K

# Some memory clean-up
K.clear_session()"""

In [ ]:
# TUNED MODEL 1: BASELINE + LR SCHEDULER + KFOLD VALIDATION + EARLY STOPPING + LeakyReLU (gridsearch for alpha) + L2 regularizer
# FAILED CHANGES VS BASELINE INDICATED WITH A HASHTAG

from keras.regularizers import l2

def build_tuned_model():
    model = keras.Sequential([
    # Convolutional layers
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01), input_shape=(156, 156, 3),kernel_regularizer=l2(0.01)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    # (CHANGE VS BASELINE) Adding another pack of Conv2D and MaxPooling2D layers
    #keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    #keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    #keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
  
    # Dense layers
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    # (CHANGE VS BASELINE) Adding Dropout
    #keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.01)),
    # (CHANGE VS BASELINE) Adding an extra dense layer
    #keras.layers.Dense(16, activation=keras.layers.LeakyReLU(alpha=0.1)),
    # (CHANGE VS BASELINE) Adding Dropout
    #keras.layers.Dropout(0.3)
    keras.layers.Dense(4, activation='softmax')])
    # Compile the model with appropriate loss function, optimizer, and metrics
    # (CHANGE VS BASELINE) Adding the optimal lr
    optim = keras.optimizers.Nadam(learning_rate=0.001) #(CHANGE VS BASELINE) optimizer = adam, nadam, sgd, rmsprop

    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_h_model():
    '''restnet blocks'''
    input_shape = (156, 156, 3)
    inputs = keras.Input(shape=input_shape)
     # Convolutional layers
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),  padding='same')(inputs)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), padding='same')(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    #  block 1
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)

    #  block 2
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    #  block 3
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    # Dense layers
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1))(x)
    x = keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1))(x)
    outputs = keras.layers.Dense(4, activation='softmax')(x)
    optim = keras.optimizers.Nadam(learning_rate=0.001) 


    # Create model
    model = keras.Model(inputs=inputs, outputs=outputs, name='h_model')

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model


hybrid_model = build_h_model()

# Train the model for 10 epochs with a batch size of 32
'''history = hybrid_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_norm, y_val_onehot))'''


In [ ]:
# K FOLD VALIDATION (5 max epochs for speeding purposes)

from sklearn.model_selection import KFold

k = 4
num_val_samples = len(X_train_norm) // k 
num_epochs = 5
tuned_all_val_losses = [] # Should add the score of each run at the end of the loop
tuned_all_val_acc = []
base_all_val_losses = []
base_all_val_acc = []
res_all_val_acc=[]
res_all_val_losses=[]

for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = X_train_norm[i * num_val_samples: (i + 1) * num_val_samples] 
    val_targets = y_train_onehot[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X_train_norm[:i * num_val_samples],
         X_train_norm[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train_onehot[:i * num_val_samples],
         y_train_onehot[(i + 1) * num_val_samples:]],
        axis=0)
    # (CHANGE VS BASELINE)Defining EarlyStopping callback
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

    baseline_model = build_baseline_model()
    tuned_model_1 = build_tuned_model()
    hybrid_model = build_h_model()
    

    # Train baseline and tuned models for 4 epochs with a batch size of 32
    print('processing baseline model')
    baseline_history = baseline_model.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=32,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
        callbacks=[early_stopping]
    )
    print('processing tuned model')
    tuned_history = tuned_model_1.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=32,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
        callbacks=[early_stopping]
    )
    print('processing hibrid model 1')
    h_history= hybrid_model.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=32,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
        callbacks=[early_stopping]
    )
    # Evaluate the kfold results for BASELINE
    base_val_loss, base_val_accuracy = baseline_model.evaluate(val_data, val_targets, verbose=0)
    base_all_val_losses.append(base_val_loss)
    base_all_val_acc.append(base_val_accuracy)

    # Evaluate the kfold results for the tuned model
    tuned_val_loss, tuned_val_accuracy = tuned_model_1.evaluate(val_data, val_targets, verbose=0)
    tuned_all_val_losses.append(tuned_val_loss)
    tuned_all_val_acc.append(tuned_val_accuracy)

    # Evaluate the kfold results for the hibrid model 1
    res_val_loss, res_val_accuracy = hybrid_model.evaluate(val_data, val_targets, verbose=0)
    res_all_val_losses.append(res_val_loss)
    res_all_val_acc.append(res_val_accuracy)

In [ ]:
print("Baseline Model KF Results:")
print("-"*len("Baseline Model Results:"))
print("Avg val_acc: {}".format(np.mean(base_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(base_all_val_losses)))
print()
print("Tuned Model KF Results:")
print("-"*len("Tuned Model KF Results:"))
print("Avg val_acc: {}".format(np.mean(tuned_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(tuned_all_val_losses)))
print()
print("ResNet Model KF Results:")
print("-"*len("ResNetmodel KF Results:"))
print("Avg val_acc: {}".format(np.mean(res_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(res_all_val_losses)))

In [ ]:

import matplotlib.pyplot as plt

def plot_comparison_acc_loss(models_compared,name_models, numepoch):
  epoch = range(1, numepoch+1)
  clr=['b','r','y','g','k','c']
  plt.figure(figsize=(10, 8))

  # Plotting the results of validation accuracy and loss for the baseline and tuned models
  plt.subplot(2, 2, 1)
  i=0
  while i<len(models_compared):
    plt.plot(epoch, models_compared[i].history['val_accuracy'], clr[i], label=name_models[i])
    i+=1
  plt.title('Validation accuracy comparison')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()
  
  i=0
  plt.subplot(2, 2, 2)
  while i<len(models_compared):
    plt.plot(epoch, models_compared[i].history['val_loss'], clr[i], label=name_models[i])
    i+=1
  plt.title('Validation loss comparison')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()
  
  i=0
  # Plotting the results of training accuracy and loss for the baseline and tuned models  
  plt.subplot(2, 2, 3)
  while i<len(models_compared):
    plt.plot(epoch, models_compared[i].history['accuracy'], clr[i], label=name_models[i])
    i+=1
  plt.title('Training accuracy comparison')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  i=0
  plt.subplot(2, 2, 4)
  while i<len(models_compared):
    plt.plot(epoch, models_compared[i].history['loss'], clr[i], label=name_models[i])
    i+=1
  plt.title('Training loss comparison')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.subplots_adjust(hspace=0.4,wspace=0.4)
  plt.show()




models_compared=[baseline_history,tuned_history,h_history]
name_models=['Baseline','Tuned','ResNet']

plot_comparison_acc_loss(models_compared,name_models, num_epochs)

### 3.5 Analyze performance of fine-tuned model

# 4. Transfer Learning

### 4.1 New data split and subsequent preprocessing adapted to transfer learning

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)

In [ ]:
#############################################
#             CONTROL PANEL (Transfer Learning)

# Sample restriction
restriction = True
address_vp_imbalance = True
proportion = 0.25

# Sample augmentation
augmentation_flip = True                       
augmentation_rotate = True

#                                           
#############################################


# safety mechanism: augmentation and restriction are not meant to be combined
if restriction == True:
    agumentation_flip = False
    augmentation_rotate = False

In [ ]:
if restriction == True:
    
    if address_vp_imbalance == True:
        restricted_sample_train, restricted_sample_labels_train = restrict_sample_vp_imb(proportion , X_train, y_train)
        
    else:
        restricted_sample_train, restricted_sample_labels_train = restrict_sample(proportion , X_train, y_train)

In [ ]:
X_train=X_train.astype('float32')
X_val=X_val.astype('float32')

# Normalization 
X_train=X_train/255.0
X_val=X_val/255.0

In [ ]:
if restriction == True:
    
    ############################
    # > RESTRICTION  SUMMARY < #
    ############################
    print("SAMPLE RESTRICTION SUMMARY")
    print("")
    print("")
    print("ORIGINAL TRAINING SAMPLE")
    print(F"Original number of instances: {len(X_train)}")
    print(f"Original instance distribution by class: \n {pd.Series(y_train).value_counts()}")
    print("")
    print("RESTRICTED TRAINING SAMPLE")
    print(f"Number of instances in restricted sample: {len(restricted_sample_train)}")
    print(f"Instance distribution by class in restricted sample: \n {pd.Series(restricted_sample_labels_train).value_counts()}")
    
    X_train = np.array(restricted_sample_train)
    y_train = np.array(restricted_sample_labels_train)
    X_val = np.array(restricted_sample_val)
    y_val = np.array(restricted_sample_labels_val)
    X_test = np.array(restricted_sample_test)
    y_test = np.array(restricted_sample_labels_test)
    
else:
    print("SAMPLE RESTRICTION WAS NOT CONDUCTED")

### 4.2 Transfer learning with VGG16

In [ ]:
# Import VGG16
vgg_model = VGG16(include_top=False, input_shape=(156, 156, 3))

# FReezing VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(vgg_model.layers[-1].output)

## Dense layers
dense_1 = layers.Dense(32, activation='relu')(flat_1)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_1)

# define new model
tl_model = Model(inputs=vgg_model.inputs, outputs=output)

# summarize
tl_model.summary()
 
# compile model
tl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
fitting = tl_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=25, verbose=1)

### 4.3 Analyze performance of transfer learning model